In [1]:
from rdflib.plugins.stores import sparqlstore
import rdflib
from rdflib import URIRef, Literal, Graph, Dataset

import serialization
import html
#import loader
import discourse
import pandas as pd
from rdflib import Graph, Namespace
from rdflib.namespace import RDF, RDFS
import io
import pydotplus
from IPython.display import display, Image
from rdflib.tools.rdf2dot import rdf2dot
from IPython.core.display import HTML


def visualize(g):
    stream = io.StringIO()
    rdf2dot(g, stream, opts = {display})
    dg = pydotplus.graph_from_dot_data(stream.getvalue())
    png = dg.create_png()

    display(Image(png))

def t2rdflibg(triples): #triples to rdflib graph
    g = Graph()
    for t in triples:
        g.add(t)
    return g


def nan2None(value):
    if pd.isnull(value) :
        return None
    else:
        return value


def get_data_rows(filename):
    data_rows_df = pd.read_csv(filename)
    data_rows=[]
    for i,r in data_rows_df.iterrows():
        r_d = {k:nan2None(v) for k,v in dict(r).items()}

        data_rows.append(r_d)
    del data_rows_df
    return data_rows

serial_key = { "Business Domain" : "ModelDomain", 
               "Model" : "Model", 
               "ModelType" : "ModelType", 
               "Entity" : "Class", 
               "Attribute" : "Attribute", 
               "DataType" : "DataType", 
               "Context" : "Context", 
               "SubContext" : "SubContext", 
               "Relationship" : "Relationship", 
               "Entity From" : "FromClass",
               "Attribute From" : "FromAttribute", 
               "Cardinality From" : "FromCardinality",
               "Entity To" : "ToClass",
               "Attribute To" : "ToAttribute",
               "Cardinality To" : "ToCardinality"
}

jena = sparqlstore.SPARQLUpdateStore("http://localhost:3030/models/query",context_aware=True)
jena.open(("http://localhost:3030/models/query", "http://localhost:3030/models/update"))

# Create a Dataset
ds = Dataset(store=jena, default_union=True, default_graph_base="http://base.raw")
# Define a named graph within the dataset, this will contain all nodes contained within the graph referenced, or none if it doesn't exist yet.
sg = ds.graph(URIRef("http://config"))
master_g = ds.graph(URIRef("http://master"))
discourse_g = ds.graph(URIRef("http://discourse"))



* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation
* Owlready2 * WARNING: ObjectProperty http://www.semanticweb.org/tomk/ontologies/2022/11/serialization#MappingMetaTarget belongs to more than one entity types: [owl.AnnotationProperty, owl.ObjectProperty, owl.topObjectProperty]; I'm trying to fix it...
* Owlready2 * Running HermiT...
    java -Xmx2000M -cp /usr/local/python/py36/lib/python3.6/site-packages/owlready2/hermit:/usr/local/python/py36/lib/python3.6/site-packages/owlready2/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////var/folders/rd/q3bhk6_n56x75m05pp2sq7dr0000gn/T/tmp5ab7n9na


Serialization.Meta [Serialization.SerializationMetaData]
Serialization.Mapping [Serialization.SerializationSpecs]
Serialization.BatchDefinition [Serialization.SerializationSpecs]
Serialization.SerializationSpecs [Serialization.SerializationMetaData]
Serialization.BatchNode [Serialization.DataLoad]
Serialization.DataLoad [Serialization.SerializationMetaData]
Serialization.SerializationMetaData [owl.Thing]
Serialization.DataPropertyMapping [Serialization.Mapping]
Serialization.MetaDataProperty [Serialization.Meta]
Serialization.EntityMapping [Serialization.Mapping]
Serialization.MetaClass [Serialization.Meta]
Serialization.JobNode [Serialization.DataLoad]
Serialization.MetaProperty [Serialization.Meta]
Serialization.PropertyMapping [Serialization.Mapping]
Serialization.RowNode [Serialization.DataLoad]
Serialization.Serialization [Serialization.SerializationSpecs]


* Owlready2 * HermiT took 0.7343430519104004 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [2]:
count_q = """SELECT count(*) ?g
{
    { ?s ?p ?o } UNION { GRAPH ?g { ?s ?p ?o } }
    
}
GROUP BY ?g"""

In [3]:
list(ds.query(count_q))

[(rdflib.term.Literal('644', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')),
  None),
 (rdflib.term.Literal('207', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')),
  rdflib.term.URIRef('http://config')),
 (rdflib.term.Literal('13775', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')),
  rdflib.term.URIRef('http://master'))]

In [4]:
count_q = """SELECT count(*) ?g
{
    { ?s ?p ?o } UNION { GRAPH ?g { ?s ?p ?o } }
    
}
GROUP BY ?g"""
list(master_g.query(count_q))

[(rdflib.term.Literal('13775', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')),
  None)]

In [5]:
# Load the serialisation configuration into its own named graph partition (fuseki)
sg.parse("EntityAttributeRecord.rdf") # does this affect the database? ... Yes
S = serialization.Serialization(sg, "EntityAttributeRecord")

In [6]:
list(ds.query(count_q))

[(rdflib.term.Literal('644', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')),
  None),
 (rdflib.term.Literal('207', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')),
  rdflib.term.URIRef('http://config')),
 (rdflib.term.Literal('13775', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')),
  rdflib.term.URIRef('http://master'))]

In [7]:
#master_g.update("""DROP GRAPH <http://master>""")


In [8]:
model_data = pd.read_csv(r"mdm_v11_3_pdm_additional.csv").fillna("")
model_data.columns=['Sequence', 'ModelDomain', 'Model', 'ModelType', 'Class',
       'ClassDescription', 'Attribute', 'AttributeDescription', 'DataType',
       'Context', 'SubContext', 'Nulls', 'Is PK']

In [9]:
def key_filter(keys,xlist=None):
    s_keys = sorted(keys)
    if xlist is not None:
        if not tuple([k for k in s_keys if k not in xlist]) == tuple():
            s_keys = tuple([k for k in s_keys if k not in xlist])
        else:
            s_keys = tuple(s_keys)

    else:
        s_keys = tuple([k for k in s_keys])
    return s_keys

from functools import partial
from collections import Counter
fkf = partial(key_filter, xlist=["LANG_TP_CD"])

In [10]:
raw_keys = dict(model_data[model_data['Is PK']=="Yes"].groupby("Class")['Attribute'].agg(fkf))
[(k,v) for k,v in Counter(raw_keys.values()).items() if len(k)>1]

[(('BKTHASH', 'RECNO', 'SRCRECNO'), 1),
 (('CMPSEQNO', 'RECNO'), 1),
 (('SRCID', 'SRCRECNO'), 1),
 (('CITY_NAME',
   'GIVEN_NAME_ONE',
   'LAST_NAME',
   'P_CITY_NAME',
   'P_GIVEN_NAME_ONE',
   'P_LAST_NAME'),
  1)]

In [11]:
# Contents here block the inversion of the table/pk mapping dict
print([(k,v) for k,v in Counter(raw_keys.values()).items() if v>1])
# FOR IBM products, sharing the same PK implies parent/sub-class relationships (though there's)
# no systematic method for determining which one is the parent.
sorted([(k,v) for k,v in raw_keys.items() if v in [k for k,v in Counter(raw_keys.values()).items() if v>1]], key=lambda x : x[1])
manual_key_masters = { ('CONT_ID',) : 'CONTACT', 
                       ('HOLDING_ID',): 'HOLDING', 
                      ('LOCATION_GROUP_ID',) : 'LOCATIONGROUP', 
                      ('PAYMENT_SOURCE_ID', ): 'PAYMENTSOURCE',
                      ('PPREF_ID',) : 'PRIVPREF',
                      ('PRODUCT_ID',) : 'PRODUCT', 
                      ('SUSPECT_ID',) : 'SUSPECT' , 
                      ('LANG_TP_CD', ) : 'CDLANGTP'
                     }

key_map = {**{v:k for k,v in raw_keys.items()}, **manual_key_masters}


[(('LOCATION_GROUP_ID',), 3), (('PAYMENT_SOURCE_ID',), 4), (('CONT_ID',), 6), (('PRODUCT_ID',), 5), (('HOLDING_ID',), 3), (('PPREF_ID',), 3), (('SUSPECT_ID',), 2)]


In [12]:
def att_link(row):
    attribute=row['Attribute']
    table = row['Class']
    t_att = tuple([attribute])
    t_val = key_map.get(t_att)
    relationship = str(table) + " --> " + str(t_val)
    if t_val is not None and t_val != table:
        return pd.Series((relationship, table, attribute, t_val, t_att[0]))
    else:
        return pd.Series((None, None, None, None, None))
    

In [13]:
model_data[['Relationship','FromClass','FromAttribute','ToClass','ToAttribute']] = model_data.apply(att_link, axis=1)
model_data

,Sequence,ModelDomain,Model,ModelType,Class,ClassDescription,Attribute,AttributeDescription,DataType,Context,SubContext,Nulls,Is PK,Relationship,FromClass,FromAttribute,ToClass,ToAttribute
0,0,IBM MDM,IBM MDM Physical Datamodel v11.3_extract_1,Physical,ADDACTIONTYPE,The ADDACTIONTYPE table identifies an action t...,ADD_ACTION_ID,"A unique, system-generated key that identifies...",BIGINT,,,Not Null,Yes,None,None,None,None,None
1,1,IBM MDM,IBM MDM Physical Datamodel v11.3_extract_1,Physical,ADDACTIONTYPE,The ADDACTIONTYPE table identifies an action t...,MATCH_RELEV_TP_CD,Identifies the CONTACT match relevancies - sco...,BIGINT,,,Not Null,No,ADDACTIONTYPE --> CDMATCHRELEVTP,ADDACTIONTYPE,MATCH_RELEV_TP_CD,CDMATCHRELEVTP,MATCH_RELEV_TP_CD
2,2,IBM MDM,IBM MDM Physical Datamodel v11.3_extract_1,Physical,ADDACTIONTYPE,The ADDACTIONTYPE table identifies an action t...,SUSP_REASON_TP_CD,Describes the critical data that was considere...,BIGINT,,,Not Null,No,ADDACTIONTYPE --> CDSUSPECTREASONTP,ADDACTIONTYPE,SUSP_REASON_TP_CD,CDSUSPECTREASONTP,SUSP_REASON_TP_CD
3,3,IBM MDM,IBM MDM Physical Datamodel v11.3_extract_1,Physical,ADDACTIONTYPE,The ADDACTIONTYPE table identifies an action t...,ORG_TP_CD,Identifies the classification of the organizat...,BIGINT,,,Null,No,ADDACTIONTYPE --> CDORGTP,ADDACTIONTYPE,ORG_TP_CD,CDORGTP,ORG_TP_CD
4,4,IBM MDM,IBM MDM Physical Datamodel v11.3_extract_1,Physical,ADDACTIONTYPE,The ADDACTIONTYPE table identifies an action t...,PERSON_ORG_CODE,Indicates the type of CONTACT - person or orga...,CHAR(1),,,Not Null,No,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1939,,IBM MDM,IBM MDM Physical Datamodel v11.3_extract_1,Physical,USERTABLE,A type of PERSON that is allowed to use the ap...,USER_ID,The unique user ID to identify the system user.\,VARCHAR(30),,,Not Null,No,None,None,None,None,None
1940,,IBM MDM,IBM MDM Physical Datamodel v11.3_extract_1,Physical,USERTABLE,A type of PERSON that is allowed to use the ap...,PASSWORD,The password used by the user to authenticate ...,VARCHAR(20,,,Null,No,None,None,None,None,None
1941,,IBM MDM,IBM MDM Physical Datamodel v11.3_extract_1,Physical,USERTABLE,A type of PERSON that is allowed to use the ap...,EXPIRY_DT,The date that the user is no longer valid.,TIMESTAMP,,,Null,No,None,None,None,None,None
1942,,IBM MDM,IBM MDM Physical Datamodel v11.3_extract_1,Physical,USERTABLE,A type of PERSON that is allowed to use the ap...,LAST_UPDATE_DT,"When a record is added or updated, this field ...",TIMESTAMP,,,Not Null,No,None,None,None,None,None


In [14]:
rows = [dict({rk:html.escape(str(rv)) for rk, rv in r.items()}) for i,r in model_data.iterrows()]

In [15]:
# Load Process Starts Here

temp_master = Graph() # Take offline copy of existing mastered triple set
delta_master = Graph()
row_triples=[]
for t in master_g:
    temp_master.add(t)

# Cycle over data rows, and using the appropriate Serialization, convert to a set of
# Mastered triples, ready to add to the master dataset.
e=0
f=0
e_mappings={}
for row in rows:
    e=e+1
    new_triples, e_mappings = S.extract_raw_triples(row, e_mappings)
    row_triples.extend (new_triples)
    
len(e_mappings)

2534

In [16]:
len(set(row_triples))

13775

In [17]:
row_triples = set(S.master_triples(temp_master, row_triples))
print(len(row_triples))


13775


In [18]:
gid = master_g._Graph__get_identifier()
quads = [(s,p,o,gid) for s,p,o in row_triples]

In [19]:
ds.addN(quads)

In [20]:
len(set(master_g))

13775

In [21]:
results = master_g.query("""select count(*) ?p { ?s ?p ?o.} group by ?p""")



In [22]:
r_df = pd.DataFrame(results)
r_df['class'] = r_df.apply(lambda x : str(x[1]).split("#")[-1], axis=1)
r_df

,0,1,class
0,371,http://www.semanticweb.org/tomk/ontologies/202...,RelationshipFromAttribute
1,371,http://www.semanticweb.org/tomk/ontologies/202...,RelationshipToClass
2,2538,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,type
3,2534,http://www.semanticweb.org/tomk/ontologies/202...,UniqueIdentifier
4,371,http://www.semanticweb.org/tomk/ontologies/202...,RelationshipToAttribute
5,2534,http://www.w3.org/2000/01/rdf-schema#label,label
6,371,http://www.semanticweb.org/tomk/ontologies/202...,RelationshipFromClass
7,2750,http://www.semanticweb.org/tomk/ontologies/202...,Contains
8,1932,http://www.semanticweb.org/tomk/ontologies/202...,DataType
9,3,http://www.semanticweb.org/tomk/ontologies/202...,ModelType


In [23]:
assert False

AssertionError: 

In [24]:


qr = master_g.query("""select ?s ?p ?o where 
    {
    { ?s ?p ?o. 
      VALUES ?p { datam:Contains }.
    }
    UNION
        { ?s ?p ?o. 
        ?s ?q ?x.
      VALUES ?q { datam:Contains }.
      VALUES ?p { rdfs:label }.
    }
    UNION
        { ?x ?q ?s. 
        ?s ?p ?o.
      VALUES ?q { datam:Contains }.
      VALUES ?p { rdfs:label }.
    }
    UNION
        { ?s ?p ?o. 
        ?s ?q ?x.
      VALUES ?q { datam:Contains }.
      VALUES ?p { rdf:type }.
    }

    UNION
        { ?x ?q ?s. 
        ?s ?p ?o.
      VALUES ?q { datam:Contains }.
      VALUES ?p { rdf:type }.
    }
    
    }
    """)



In [25]:
pd.DataFrame(qr)

,0,1,2
0,http://www.semanticweb.org/tomk/ontologies/202...,http://www.semanticweb.org/tomk/ontologies/202...,http://www.semanticweb.org/tomk/ontologies/202...
1,http://www.semanticweb.org/tomk/ontologies/202...,http://www.semanticweb.org/tomk/ontologies/202...,http://www.semanticweb.org/tomk/ontologies/202...
2,http://www.semanticweb.org/tomk/ontologies/202...,http://www.semanticweb.org/tomk/ontologies/202...,http://www.semanticweb.org/tomk/ontologies/202...
3,http://www.semanticweb.org/tomk/ontologies/202...,http://www.semanticweb.org/tomk/ontologies/202...,http://www.semanticweb.org/tomk/ontologies/202...
4,http://www.semanticweb.org/tomk/ontologies/202...,http://www.semanticweb.org/tomk/ontologies/202...,http://www.semanticweb.org/tomk/ontologies/202...
...,...,...,...
13752,http://www.semanticweb.org/tomk/ontologies/202...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.semanticweb.org/tomk/ontologies/202...
13753,http://www.semanticweb.org/tomk/ontologies/202...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.semanticweb.org/tomk/ontologies/202...
13754,http://www.semanticweb.org/tomk/ontologies/202...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.semanticweb.org/tomk/ontologies/202...
13755,http://www.semanticweb.org/tomk/ontologies/202...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.semanticweb.org/tomk/ontologies/202...


In [26]:
visualize(t2rdflibg(qr))

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.0281548 to fit

